#### Import libs

In [1]:
%matplotlib qt
import numpy as np
import scipy.signal as signal
from numpy.fft import fft, ifft
import matplotlib.pyplot as plt
from expyfun.io import read_wav
import mne

#### Define Filtering Functions

we have to use causal filter

In [2]:
# %% Define Filtering Functions

def butter_highpass(cutoff, fs, order=1):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = signal.butter(order, normal_cutoff, btype='high', analog=False)
    return b, a


def butter_highpass_filter(data, cutoff, fs, order=1):
    b, a = butter_highpass(cutoff, fs, order=order)
    y = signal.lfilter(b, a, data)
    return y


def butter_lowpass(cutoff, fs, order=1):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = signal.butter(order, normal_cutoff, btype='low', analog=False)
    return b, a


def butter_lowpass_filter(data, cutoff, fs, order=1):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = signal.lfilter(b, a, data)
    return y


def butter_bandpass(lowcut, highcut, fs, order=1):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = signal.butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=1):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = signal.lfilter(b, a, data)
    return y


### Basic Experiment Parameters

In [3]:
# %% Parameters
# Anlysis
is_click = True # if derive click ABR
is_ABR = True # if derive only ABR channels
# Stim param
stim_fs = 48000 # stimulus sampling frequency
t_click = 60 # click trial length
click_rate = 40
# EEG param
eeg_n_channel = 2 # total channel of ABR
eeg_fs = 10000 # eeg sampling frequency
eeg_f_hp = 1 # high pass cutoff

### Load EEG data

In [4]:
#EEG
eeg_vhdr = '/Users/tongshan/Documents/ABR/data/pilot_10_20250716-selected/pilot_10_full.vhdr'
eeg_raw = mne.io.read_raw_brainvision(eeg_vhdr, preload=True)
events, event_dict = mne.events_from_annotations(eeg_raw)

Extracting parameters from /Users/tongshan/Documents/ABR/data/pilot_10_20250716-selected/pilot_10_full.vhdr...
Setting channel info structure...
Reading 0 ... 14457599  =      0.000 ...  1445.760 secs...
Used Annotations descriptions: ['New Segment/', 'Stimulus/S  1', 'Stimulus/S  2', 'Stimulus/S  4', 'Stimulus/S  8']


### Pick ABR channels

In [5]:
# Select ABR channels
eeg_raw.pick_channels(['Plus_R', 'Minus_R', 'Plus_L', 'Minus_L'])

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


<RawBrainVision | pilot_10_full.eeg, 4 x 14457600 (1445.8 s), ~441.2 MiB, data loaded>

### get ABR channels with reference

In [6]:
# Right ear: Plus_R - Minus_R
# Left ear: Plus_L - Minus_L
data_R = eeg_raw.get_data(picks=eeg_raw.ch_names[0]) - eeg_raw.get_data(picks=eeg_raw.ch_names[1]) # Right ear
data_L = eeg_raw.get_data(picks=eeg_raw.ch_names[2]) - eeg_raw.get_data(picks=eeg_raw.ch_names[3]) # Left ear
data = np.vstack((data_R, data_L)) # Combine channels
data /= 100 # Scale data to microvolts

# make info for RawArray
info = mne.create_info(ch_names=["EP1","EP2"], sfreq=eeg_raw.info['sfreq'], ch_types='eeg') 
eeg_raw_ref = mne.io.RawArray(data, info)

Creating RawArray with float64 data, n_channels=2, n_times=14457600
    Range : 0 ... 14457599 =      0.000 ...  1445.760 secs
Ready.


### EEG Preprocessing

In [7]:
# high pass filter
eeg_raw_ref._data = butter_highpass_filter(eeg_raw_ref._data, eeg_f_hp, eeg_fs)
# Notch filter
notch_freq = np.arange(60, 540, 180)
notch_width = 5
for nf in notch_freq:
    bn, an = signal.iirnotch(nf / (eeg_fs / 2.), float(nf) / notch_width)
    eeg_raw_ref._data = signal.lfilter(bn, an, eeg_raw_ref._data)

### Epoching

In [8]:
# Epoching click
print('Epoching EEG click data...')
epochs_click = mne.Epochs(eeg_raw_ref, events, tmin=0,
                            tmax=(t_click - 1/stim_fs + 1),
                            event_id=1, baseline=None,
                            preload=True, proj=False)
epoch_click = epochs_click.get_data()

Epoching EEG click data...
Not setting metadata
75 matching events found
No baseline correction applied
Using data from preloaded Raw for 75 events and 610001 original time points ...
0 bad epochs dropped


### Load click wave files and convert to pulse trains

In [ ]:
# Load click wave file
n_epoch_click = 5
x_in = np.zeros((n_epoch_click, int(t_click * eeg_fs)), dtype=float)
for ei in range(n_epoch_click):
    stim, fs_stim = read_wav('/Users/tongshan/Documents/ABR/present_files/click/' +
                                'click{0:03d}'.format(ei) + '.wav')
    stim_abs = np.abs(stim)
    click_times = [(np.where(np.diff(s) > 0)[0] + 1) /
                    float(fs_stim) for s in stim_abs] # Read click event
    click_inds = [(ct * eeg_fs).astype(int) for ct in click_times]
    x_in[ei, click_inds] = 1 # generate click train as x_in

2025-07-22 14:41:41,588 - INFO    - Read WAV file with 1 channel and 2880000 samples (format int16)
2025-07-22 14:41:41,604 - INFO    - Read WAV file with 1 channel and 2880000 samples (format int16)
2025-07-22 14:41:41,616 - INFO    - Read WAV file with 1 channel and 2880000 samples (format int16)
2025-07-22 14:41:41,629 - INFO    - Read WAV file with 1 channel and 2880000 samples (format int16)
2025-07-22 14:41:41,640 - INFO    - Read WAV file with 1 channel and 2880000 samples (format int16)
2025-07-22 14:41:41,651 - INFO    - Read WAV file with 1 channel and 2880000 samples (format int16)
2025-07-22 14:41:41,664 - INFO    - Read WAV file with 1 channel and 2880000 samples (format int16)
2025-07-22 14:41:41,675 - INFO    - Read WAV file with 1 channel and 2880000 samples (format int16)
2025-07-22 14:41:41,691 - INFO    - Read WAV file with 1 channel and 2880000 samples (format int16)
2025-07-22 14:41:41,705 - INFO    - Read WAV file with 1 channel and 2880000 samples (format int16)


### FFT of click trains (`x_in`) and EEG (`x_out`)

In [ ]:
# Get x_out
len_eeg = int(eeg_fs * t_click)
x_out = np.zeros((n_epoch_click, 2, len_eeg))
for i in range(n_epoch_click):
    x_out_i = epoch_click[i, :, 0:int(eeg_fs*t_click)]
    x_out[i, :, :] = mne.filter.resample(x_out_i, eeg_fs, eeg_fs)
x_out = np.mean(x_out, axis=1) # average the two channels

### Cross Correlation in frequency domain

In [11]:
# Derive ABR
print('Deriving ABR through cross-correlation...')
t_start, t_stop = -200e-3, 600e-3 # ABR showed range
# FFT
x_in_fft = fft(x_in, axis=-1)
x_out_fft = fft(x_out, axis=-1)

Deriving ABR through cross-correlation...


In [12]:
# Cross Correlation in frequency domain
cc = np.real(ifft(x_out_fft * np.conj(x_in_fft)))
abr = np.mean(cc, axis=0) # average across 10 trials
abr /= (click_rate*t_click) # real unit value

### Concatanate the derived response
Since the impulse response is circular, we can concatenate the last 200 ms to be as -200 ms before onset

In [13]:
# Concatanate click ABR response as [-200, 600] ms lag range
abr_response = np.concatenate((abr[int(t_start*eeg_fs):],
                                abr[0:int(t_stop*eeg_fs)]))
# generate time vector
lags = np.arange(start=t_start*1000, stop=t_stop*1000, step=1e3/eeg_fs)

### Plot the response

In [14]:
# Plot ABR
plt.figure()
plt.plot(lags, abr_response)
plt.xlim([-20, 60])
plt.xlabel('Lag (ms)')
plt.ylabel('Amplitude (uV)')
plt.title('ABR response')

Text(0.5, 1.0, 'ABR response')